**Proyecto:** Bogotá Visible - Datos al Ecosistema 2025


**Objetivo:** Normalizar, limpiar y cruzar espacialmente 5 fuentes de datos oficiales para alimentar la aplicación de Streamlit.
**Salida:** Archivos GeoJSON optimizados (EPSG:4326) en la carpeta `DATOS_LIMPIOS`.

In [1]:
# 2. Importación de Librerías
import pandas as pd
import geopandas as gpd
import numpy as np

## 1. Procesamiento de Colegios
**Fuente:** Directorio Único de Establecimientos (Secretaría de Educación).
**Transformación:** Limpieza de coordenadas lat/lon y selección de columnas clave.

In [2]:
url_colegios = "https://github.com/andres-fuentex/CONCURSO/raw/main/DATOS_BRUTOS/directorio-unico-de-establecimientos-06.25.xlsx"
cols = pd.read_excel(url_colegios)

# Selección y Renombre
df_cols = cols[["NOMBRE_ESTABLECIMIENTO_EDUCATIVO","SECTOR","CALENDARIO","COORDENADA LONGITUD (X)","COORDENADA LATITUD (Y)"]].copy()
df_cols.rename(columns={
    "NOMBRE_ESTABLECIMIENTO_EDUCATIVO": "nombre",
    "SECTOR": "sector",
    "CALENDARIO": "calendario",
    "COORDENADA LONGITUD (X)": "longitud",
    "COORDENADA LATITUD (Y)": "latitud"  
}, inplace=True)

# Limpieza de Coordenadas (Función reutilizable)
def limpiar_coord(val):
    if pd.isna(val): return np.nan
    return str(val).replace('\t', '').replace(',', '.').strip()

df_cols["longitud"] = pd.to_numeric(df_cols["longitud"].apply(limpiar_coord), errors='coerce')
df_cols["latitud"] = pd.to_numeric(df_cols["latitud"].apply(limpiar_coord), errors='coerce')

# Convertir a GeoDataFrame y Exportar
gdf_colegios = gpd.GeoDataFrame(
    df_cols.dropna(subset=["longitud", "latitud"]),
    geometry=gpd.points_from_xy(df_cols.dropna()["longitud"], df_cols.dropna()["latitud"]),
    crs="EPSG:4326"
)

# Exportar
gdf_colegios.to_file("dim_colegios.geojson", driver="GeoJSON")

##  2. Procesamiento de Áreas POT
**Fuente:** Área de Actividad (Planeación Distrital).
**Transformación:** Mapeo de categorías técnicas a lenguaje ciudadano (Ej: 'AAE' -> 'Empresarial').

In [4]:
url_pot = "https://github.com/andres-fuentex/CONCURSO/raw/main/DATOS_BRUTOS/areaactividad.zip"
gdf_area = gpd.read_file(url_pot)

# Selección
gdf_area = gdf_area[["NOMBRE_ARE", "geometry"]].rename(columns={"NOMBRE_ARE": "nombre_area"})

# Lógica de Negocio: Simplificación de Usos
mapeo_pot = {
    'Área de Actividad de Proximidad - AAP - Receptora de soportes urbanos': 'Residencial Mixto',
    'Área de Actividad de Proximidad - AAP- Generadora de soportes urbanos': 'Comercio y Servicios Locales',
    'Área de Actividad Estructurante - AAE - Receptora de actividades económicas': 'Empresarial e Industrial',
    'Área de Actividad Estructurante - AAE - Receptora de vivienda de interés social': 'Alta Densidad / VIS',
    'Área de Actividad Grandes Servicios Metropolitanos - AAGSM': 'Grandes Dotacionales',
    'Plan Especial de Manejo y Protección -PEMP BIC Nacional...': 'Histórico y Patrimonial'
}

# Aplicamos mapeo (usando .map para velocidad)
# Nota: Ajustamos la llave larga del PEMP con un startswith o lambda si es necesario, 
# pero si el string es exacto, el map funciona.
gdf_area['uso_pot_simplificado'] = gdf_area['nombre_area'].apply(
    lambda x: next((v for k, v in mapeo_pot.items() if k in str(x)), 'Otro / Sin Clasificar')
)

# Exportar
gdf_area.to_crs("EPSG:4326").to_file("dim_area.geojson", driver="GeoJSON")

## 3. Seguridad por Localidad (Algoritmo Top 3)
**Fuente:** Delitos de Alto Impacto - DAI (Secretaría de Seguridad).
**Transformación:** Cálculo de ranking de delitos por localidad para generar contexto narrativo.

In [5]:
url_loc = "https://github.com/andres-fuentex/CONCURSO/raw/main/DATOS_BRUTOS/dai_shp.zip"
gdf_loc = gpd.read_file(url_loc)

# Diccionario de Delitos (Columnas 2024 o 2025 según tu dataset)
cols_delitos = {
    'CMHR24CONT': 'Hurto Residencias',
    'CMHC24CONT': 'Hurto Comercio',
    'CMHA24CONT': 'Hurto Autos',
    'CMHM24CONT': 'Hurto Motos',
    'CMHB24CONT': 'Hurto Bicicletas',
    'CMH24CONT':  'Homicidios'
}

def obtener_top_3(row):
    """Calcula los 3 delitos más frecuentes en la fila"""
    datos = row[list(cols_delitos.keys())].rename(index=cols_delitos)
    top = datos.sort_values(ascending=False).head(3)
    top = top[top > 0] # Eliminar ceros
    if top.empty: return "Sin datos significativos"
    return ", ".join([f"{idx}" for idx, val in top.items()])

# Aplicar lógica
gdf_loc['top_3_delitos'] = gdf_loc.apply(obtener_top_3, axis=1)
gdf_loc = gdf_loc[['CMNOMLOCAL', 'CMIULOCAL', 'top_3_delitos', 'geometry']]
gdf_loc.rename(columns={'CMNOMLOCAL': 'nombre_localidad', 'CMIULOCAL': 'id_localidad'}, inplace=True)

# Exportar
gdf_loc.to_crs("EPSG:4326").to_file("dim_localidad.geojson", driver="GeoJSON")

##  4. Infraestructura de Movilidad
**Fuente:** Estaciones de Transmilenio (Datos Abiertos).
**Transformación:** Conversión de CSV a GeoJSON.

In [6]:
url_tm = "https://github.com/andres-fuentex/CONCURSO/raw/main/DATOS_BRUTOS/estaciones-de-transmilenio.csv"
df_tm = pd.read_csv(url_tm, sep=';', encoding='utf-8', on_bad_lines='skip')

gdf_tm = gpd.GeoDataFrame(
    df_tm,
    geometry=gpd.points_from_xy(df_tm['coord_x'], df_tm['coord_y']),
    crs="EPSG:4326"
)

gdf_tm = gdf_tm[['Nombre Estación', 'Troncal Estación', 'geometry']].rename(columns={
    'Nombre Estación': 'nombre_estacion',
    'Troncal Estación': 'troncal'
})

gdf_tm.to_file("dim_transporte.geojson", driver="GeoJSON")

##  5. Manzanas y Estratificación
**Fuente:** Manzana Estratificación (Ideca).
**Transformación:** Cruce espacial (Spatial Join) para asignar ID de localidad a cada manzana.

In [7]:
url_manz = "https://github.com/andres-fuentex/CONCURSO/raw/main/DATOS_BRUTOS/manzanaestratificacion.zip"
gdf_manz = gpd.read_file(url_manz)

# Limpieza
gdf_manz = gdf_manz[['CODIGO_MAN', 'ESTRATO', 'geometry']].rename(columns={
    'CODIGO_MAN': 'codigo_manzana',
    'ESTRATO': 'estrato'
})
gdf_manz['estrato'] = pd.to_numeric(gdf_manz['estrato'], errors='coerce').fillna(0).astype(int)

# Cruce Espacial (Asignar Localidad)
# Nota: Aseguramos que ambas capas estén en el mismo CRS antes del cruce
if gdf_manz.crs != "EPSG:4326": gdf_manz = gdf_manz.to_crs("EPSG:4326")
# Reusamos gdf_loc que ya cargamos arriba
gdf_manz_loc = gpd.sjoin(gdf_manz, gdf_loc[['id_localidad', 'geometry']], how='inner', predicate='intersects')

# Limpieza post-cruce
gdf_final = gdf_manz_loc[['codigo_manzana', 'estrato', 'id_localidad', 'geometry']]

# Exportar
gdf_final.to_file("tabla_hechos.geojson", driver="GeoJSON")

C:\Users\andre\AppData\Local\Temp\ipykernel_23664\544599071.py:15: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4686

  gdf_manz_loc = gpd.sjoin(gdf_manz, gdf_loc[['id_localidad', 'geometry']], how='inner', predicate='intersects')


## 6. Red adscrita a salud 
**Fuente:** RASA (Red Adscrita de Salud) 
**Transformación:** Normalización de coordenadas a WGS84 para visualización web y selección de atributos clave.

In [8]:

url_salud = "https://github.com/andres-fuentex/CONCURSO/raw/main/DATOS_BRUTOS/rasa.zip"
gdf_salud = gpd.read_file(url_salud)

# Selección y Renombre
gdf_salud = gdf_salud[["RSOENTADSC", "geometry"]].rename(columns={
        "RSOENTADSC": "nombre_hospital"
})

# Normalización de Coordenadas y Exportación
gdf_salud.to_crs("EPSG:4326").to_file("dim_salud.geojson", driver='GeoJSON')


## 7. y Sistema de Parques 
**Fuente:** Inventario de parques segun POT
**Transformación:** Normalización de coordenadas a WGS84 para visualización web y selección de atributos clave.

In [9]:
url_parques = "https://github.com/andres-fuentex/CONCURSO/raw/main/DATOS_BRUTOS/parque.zip"
gdf_parques = gpd.read_file(url_parques)

# Selección y Renombre
gdf_parques = gdf_parques[["NOMBRE", "geometry"]].rename(columns={
        "NOMBRE": "nombre_parque"
})

# Normalización de Coordenadas y Exportación
# Nota: Al forzar to_crs aquí, solucionamos el problema de que no se vean en el mapa
gdf_parques.to_crs("EPSG:4326").to_file("dim_verde.geojson", driver='GeoJSON')